# Examples - DataFrame - Simple distributed joins
https://gist.github.com/mrocklin/7b3d3c1b9ed3e747aaf04ad70debc8e9

## Join Dask.DataFrame against small Pandas DataFrame

We join a small pandas dataframe against a larger dask dataframe.  This is a common-case problem.

We do this with a fake dataset.

### Create fake dataset of products and categories

In [1]:
from itertools import product

alphabet = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
products = [''.join(x) for x in product(alphabet, alphabet, alphabet, alphabet)]
len(products)

456976

In [2]:
products[:8]

['AAAA', 'AAAB', 'AAAC', 'AAAD', 'AAAE', 'AAAF', 'AAAG', 'AAAH']

In [3]:
products[-8:]

['ZZZS', 'ZZZT', 'ZZZU', 'ZZZV', 'ZZZW', 'ZZZX', 'ZZZY', 'ZZZZ']

In [4]:
import pandas as pd
import numpy as np

dimension = pd.DataFrame({'product': products, 
                          'category': np.random.randint(0, 1000, size=len(products))})
dimension.head()

,category,product
0,140,AAAA
1,276,AAAB
2,319,AAAC
3,546,AAAD
4,973,AAAE


In [5]:
def fake_data(size):
    import random
    import numpy as np
    return pd.DataFrame({'product': [''.join(random.sample(alphabet, 4)) for i in range(size)],
                         'value': np.random.exponential(100, size=size).astype(int),
                         'rating': np.random.normal(0, 1, size=size)})

In [6]:
example = fake_data(5)
example

,product,rating,value
0,CDFK,1.979967,100
1,CTSR,2.242478,187
2,MGZY,-1.340031,102
3,FROP,0.763971,25
4,IXYR,0.117272,137


In [7]:
example.merge(dimension, on='product', how='inner')

,product,rating,value,category
0,CDFK,1.979967,100,746
1,CTSR,2.242478,187,340
2,MGZY,-1.340031,102,784
3,FROP,0.763971,25,86
4,IXYR,0.117272,137,956


## Parallelize with Dask.Dataframe

We do the same thing but now in parallel with dask.dataframe.

We start on a single machine.  We'll repeat on a cluster later.

### Make a fake fact table

Normally we we would load our data in from some external source as in the following:

```python
df = dask.dataframe.read_csv('hdfs://path/to/my/data/*.csv')
```

Instead we create a `fake_data` function to produce each "file".  You can ignore this if you have data elsewhere.

In [8]:
import dask.dataframe as dd
from dask import delayed

partitions = []
for i in range(10):
    partitions.append(delayed(fake_data)(10000))
    
example = fake_data(1)

df = dd.from_delayed(partitions, example)

In [9]:
df

,product,rating,value
npartitions=10,,,
,object,float64,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [10]:
df.tail()

,product,rating,value
9995,ICAW,-0.808129,865
9996,FIUQ,1.101496,97
9997,WFDN,-0.085942,37
9998,MBCK,0.083766,3
9999,OJNY,1.914300,77


In [11]:
joined = df.merge(dimension, how='inner', on='product')
joined.tail()

,product,rating,value,category
9995,JSAP,0.376465,163,472
9996,COJZ,0.181082,112,377
9997,NBUA,3.292856,189,625
9998,DEGF,-0.337251,10,594
9999,XBHJ,0.554801,111,649


### Analyze the joined table

We'll find the top rated categories

In [12]:
joined.groupby('category').rating.mean().nlargest(10).compute()

category
212    0.292520
760    0.291430
127    0.282074
150    0.264172
456    0.235768
16     0.223945
863    0.222552
229    0.217775
317    0.212684
43     0.212373
Name: rating, dtype: float64

## Distributed computing

We connect to a cluster of workers, and repeat the experiment on a larger cluster.

In [14]:
from dask.distributed import Client, progress
e = Client()
e

Client Scheduler: tcp://127.0.0.1:42047 Dashboard: http://127.0.0.1:34669/status,Cluster Workers: 2 Cores: 2 Memory: 8.28 GB


In [15]:
dfs = [delayed(fake_data)(10000) for i in range(1000)]
example = fake_data(1)
df = dd.from_delayed(dfs, example)
df

,product,rating,value
npartitions=1000,,,
,object,float64,int64
,...,...,...
...,...,...,...
,...,...,...
,...,...,...


In [19]:
joined = df.merge(dimension, how='inner', on='product')
result = joined.groupby('category').rating.mean().nlargest(10)

future = e.compute(result)
future

<Future: status: error, key: finalize-25017c67d6b22ad7ce61af20c6f18c4e>

In [20]:
type(future)

distributed.client.Future

In [21]:
progress(future)

VBox()

In [22]:
dimension.head()

,category,product
0,140,AAAA
1,276,AAAB
2,319,AAAC
3,546,AAAD
4,973,AAAE


### Future Work

Looking at the profile plot its clear that performing the merge is taking up most of the time.  This merge is against an text column which is implemented using the Python Object dtype, which is both slow and stops multi-core work.  I suspect that by switching to a categorical or other numeric dtype we could run much much faster.